In [1]:
import geemap
import ee

In [2]:
ee.Initialize()

In [4]:
gadm_4 = ee.FeatureCollection("projects/ee-gopalpenny/assets/INDIA_SUBDISTRICT_11")

In [61]:
# gadm_4.filter(ee.Filter.inList('STATE_UT',['Bihar','Uttar Pradesh','Punjab','Haryana'])).size().getInfo()
subdists = gadm_4.filter(ee.Filter.inList('STATE_UT',['Bihar','Uttar Pradesh','Punjab','Haryana'])) #.size().getInfo()
# subdists = gadm_4.filter(ee.Filter.inList('STATE_UT',['Haryana'])) #.size().getInfo()
subdists.size().getInfo()

997

In [62]:
firms = ee.ImageCollection("FIRMS").select("T21")

firesVis = {
  'min': 325.0,
  'max': 400.0,
  'palette': ['red', 'orange', 'yellow'],
}

In [63]:
years = ee.List.sequence(2010,2015)
yr = years.get(0)

def getFires(yr):
    start = ee.Date.fromYMD(yr,11,1)
    end = ee.Date.fromYMD(yr,12,1)
    fires_november = firms.filterDate(start,end).reduce(ee.Reducer.max())
    fires_november = fires_november.set('year',yr)
    return fires_november

fires_november = ee.ImageCollection(years.map(getFires))
fires_november.first().getInfo()


{'type': 'Image',
 'bands': [{'id': 'T21_max',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'year': 2010, 'system:index': '0'}}

In [64]:

def reduce_fires(image):
    subdist_fires = image.reduceRegions(
      collection=subdists,
      reducer=ee.Reducer.count(),
      scale=1000,
    )
    subdist_fires = subdist_fires.map(lambda x: x.set('year',image.get('year')))
    return subdist_fires

fires = fires_november.map(reduce_fires).flatten()
fires.first().getInfo()


{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[83.93463125078125, 27.297372587197547],
    [83.93305270815907, 27.2947595114809],
    [83.93110851909206, 27.291861104918],
    [83.92796486649225, 27.288819979737077],
    [83.92308662086133, 27.285801185654694],
    [83.9180299523166, 27.281707716498683],
    [83.91575587488103, 27.28004003634614],
    [83.91438244925371, 27.27913931548363],
    [83.91055206630246, 27.276570814338225],
    [83.90930800709471, 27.275050338014907],
    [83.9092856991495, 27.273511917784933],
    [83.9092633911962, 27.27196901860661],
    [83.90991888600021, 27.269971390453446],
    [83.90970041545376, 27.266181140563855],
    [83.90994119371469, 27.259880442839084],
    [83.91108712037547, 27.2560188115819],
    [83.91259882491937, 27.253240847905428],
    [83.91460987398608, 27.24967804819409],
    [83.91750376777901, 27.24748862162901],
    [83.92520915925681, 27.24463919252254],
    [83.93189329603342, 27.242409641826335],
   

In [65]:
task = ee.batch.Export.table.toDrive(
    collection=fires,
    description='fires',
    fileFormat='CSV',
    selectors=['DISTRICT','NAME','STATE_UT','count','SUB_ID','year'],
    fileNamePrefix='fires_count_n_india_2010_2016',
    folder='GEE_Div',
    )
task.start()

In [33]:
m = geemap.Map()

m.setCenter(80, 20, 6)
m.addLayer(fires, firesVis, 'Firms')
m

Map(center=[20, 80], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…